A jupyter notebook to test the code generation module

In [16]:
import os
import json
import pandas as pd
from openai import OpenAI
from dotenv import load_dotenv

In [17]:
# Read the CSV file into a Pandas DataFrame
# print(os.getcwd())
calendar_data = pd.read_csv("../data/calendar_data.csv")
print(f"The number of calendar data is {len(calendar_data)}.")

The number of calendar data is 20.


In [18]:
calendar_data.iloc[0]['start']

'2024-02-06 09:00:00-00:00'

1. Loading the dataset and json
2. Prompt the dataset column and json question to gpt-4 to generate the python function
3. pass the generated funtion and dataframe into the test function
4. save the result into json-ansewer

In [57]:
question_index = 1 # for question test

with open("../data/question.json") as json_file:
    json_data = json.load(json_file)
    print(json_data[question_index])

{'question': 'How many events do I have scheduled for today?', 'answer': ''}


In [58]:
print(f"The total number of the question is {len(json_data)}.")
print(f"Selected question is: <{json_data[question_index]['question']}>.")

The total number of the question is 8.
Selected question is: <How many events do I have scheduled for today?>.


In [65]:
text = f"""
{json_data[question_index]['question']}
"""
PROMPT = f"""You are provided a Pandas dataframe named calendar_data, columns = [ID, status, summary, start, end , duration, attendees]. DataFrame calendar_data includes all you calenders.
Your task is generate python function to query this dataframe and answer the question. Output a python code by enclosing it in triple backticks. 
Today's date is '2024-03-12 13:02:30-03:20'.

The input have following columns:
- ID: meeting ID;
- status: meeting status, including the following status: cancelled, confirmed, tentative;
- summary: meeting or event topic;
- start: the start date of meeting, date format: YYYY-MM-DD hh:mm:ss.fff-zz:xx. for example "2024-02-05 12:00:00-00:00";
- end: the start date of meeting, date format: YYYY-MM-DD hh:mm:ss.fff-zz:xx, for example "2024-02-05 13:00:00-00:00";
- duration: meeting duration (second);
- attendees: people who attend the meeting delimited by the line terminator within 1 sentence.

Output is executable Python code by enclosing it in triple backticks:
```python
<your code here>
```

The input of python code is a Pandas dataframe named calendar_data, and the answer is saved in variable answer.

For example, the output have the following format:
```
import pandas as pd 
def query(calendar_data):
    return calendar_data[0]
answer = query(calendar_data)
```

Question to be resolved: {text} 
"""

In [60]:
print(PROMPT)

You are provided a Pandas dataframe named calendar_data, columns = [ID, status, summary, start, end , duration, attendees]. DataFrame calendar_data includes all you calenders.
Your task is generate python function to query this dataframe and answer the question. Output a python code block without any prefix instructionn and explaination. 
Today's date is '2024-03-12 13:02:30-03:20'.

The input have following columns:
- ID: meeting ID;
- status: meeting status, including the following status: cancelled, confirmed, tentative;
- summary: meeting or event topic;
- start: the start date of meeting, date format: YYYY-MM-DD hh:mm:ss.fff-zz:xx. for example "2024-02-05 12:00:00-00:00";
- end: the start date of meeting, date format: YYYY-MM-DD hh:mm:ss.fff-zz:xx, for example "2024-02-05 13:00:00-00:00";
- duration: meeting duration (second);
- attendees: people who attend the meeting delimited by the line terminator within 1 sentence.


Output is executable Python code by enclosing it in triple 

In [61]:

MODEL = "gpt-4"
# MODEL = "gpt-3.5-turbo"

def get_completion(MODEL, PROMPT):
    client = OpenAI(
        # This is the default and can be omitted
        api_key=os.environ.get("OPENAI_API_KEY"),
    )

    chat_completion = client.chat.completions.create(
        messages=[
            {
                "role": "user",
                "content": PROMPT,
            }
        ],
        model=MODEL,
    )

    # verify the output
    return chat_completion.choices[0].message.content

llm_reply_with_code = get_completion(MODEL, PROMPT)
print(llm_reply_with_code)

```python
import pandas as pd 

def query(calendar_data):
    today = pd.to_datetime('2024-03-12 13:02:30-03:20').normalize()
    mask = (pd.to_datetime(calendar_data['start']).dt.normalize() == today) & (calendar_data['status'] == 'confirmed')
    return calendar_data[mask].shape[0]

answer = query(calendar_data)
```
This python code will first convert the input date '2024-03-12 13:02:30-03:20' to a timestamp and find the date part (normalize function). Then it will filter the calendar_data DataFrame on two conditions: 'start' date equals to today and 'status' equals to 'confirmed'. Finally, the number of these filtered rows are returned.


In [62]:
import re

_python_code_re_pattern = "```python\n(.*?)```"
llm_reply_without_code = re.sub(
            _python_code_re_pattern, "", llm_reply_with_code, flags=re.DOTALL
            )
python_code_list = re.findall(_python_code_re_pattern, llm_reply_with_code, re.DOTALL)


In [63]:
python_code_list[0]

"import pandas as pd \n\ndef query(calendar_data):\n    today = pd.to_datetime('2024-03-12 13:02:30-03:20').normalize()\n    mask = (pd.to_datetime(calendar_data['start']).dt.normalize() == today) & (calendar_data['status'] == 'confirmed')\n    return calendar_data[mask].shape[0]\n\nanswer = query(calendar_data)\n"

In [64]:
exec(python_code_list[0])
answer

0